In [140]:
import geopandas as gpd
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import statsmodels.api as sm
import shapely
pd.set_option('display.max_columns', None)

In [29]:
shorelines = gpd.read_file("Data for testing/NZCCDv1.shp")
shorelines

,Region,Site,Digitiser,Scale,Notes,Source,CPS,Proxy,Photoscale,Georef_ER,Pixel_Er,Total_UNCY,USDate,SHLength,Date,ID,geometry
0,Auckland,KarekareBethells,MW,2000,tod,RL,4,1,40000,5.03,1.416425,5.620681,01/02/2004,1.265403,2004-01-02,0.0,"LINESTRING Z (1728907.500 5916213.248 0.000, 1..."
1,Auckland,KarekareBethells,MW,2000,tod,RL,4,1,40000,5.03,1.416425,5.620681,01/02/2004,0.307010,2004-01-02,1.0,"LINESTRING Z (1729067.838 5914779.733 0.000, 1..."
2,Auckland,KarekareBethells,MW,2000,tod (landward),RL,4,1,40000,5.03,1.416425,5.620681,01/02/2004,0.255243,2004-01-02,2.0,"LINESTRING Z (1729647.881 5911983.994 0.000, 1..."
3,Auckland,KarekareBethells,MW,2000,tod,RL,3,1,40000,5.03,1.416425,5.314890,01/02/2004,1.107251,2004-01-02,3.0,MULTILINESTRING Z ((1728131.079 5917218.888 0....
4,Auckland,KarekareBethells,MW,2000,tod,RL,4,1,40000,5.03,1.416425,5.620681,01/02/2004,0.770375,2004-01-02,4.0,MULTILINESTRING Z ((1729643.228 5912906.682 0....
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19661,West Coast,WoodpeckerBay,MD,1000,EdgeVeg,RL,2,1,18800,4.00,0.630693,3.736680,11/18/1948,1.191368,1948-11-18,19661.0,"LINESTRING Z (1467237.671 5348414.193 0.000, 1..."
19662,Otago,Oamaru,TK,1500,Top of cliff,Max,3,2,0,2.02,0.412236,2.278429,04/04/2018,0.650952,2018-04-04,19662.0,"LINESTRING Z (1444386.944 5008247.367 0.000, 1..."
19663,Otago,Oamaru,TK,1500,Top of cliff,Max,3,2,0,2.02,0.412236,2.278429,04/04/2018,0.179434,2018-04-04,19663.0,"LINESTRING Z (1444860.629 5008686.925 0.000, 1..."
19664,Otago,Oamaru,TK,1500,Top of cliff,Max,3,2,0,2.02,0.412236,2.278429,04/04/2018,0.174825,2018-04-04,19664.0,"LINESTRING Z (1444996.286 5008822.252 0.000, 1..."


In [3]:
transects = gpd.read_file("Data for testing/Unique_ID_transects/NorthIsland_transects.shp")
transects

,Unique_ID,geometry
0,1.005226e+11,"LINESTRING Z (1680443.328 5669799.874 0.000, 1..."
1,1.005226e+11,"LINESTRING Z (1680451.296 5669793.831 0.000, 1..."
2,1.005226e+11,"LINESTRING Z (1680459.263 5669787.788 0.000, 1..."
3,1.005227e+11,"LINESTRING Z (1680467.271 5669781.803 0.000, 1..."
4,1.005227e+11,"LINESTRING Z (1680475.694 5669776.413 0.000, 1..."
...,...,...
139133,1.004662e+11,"LINESTRING Z (1799080.909 5395344.095 0.000, 1..."
139134,1.004662e+11,"LINESTRING Z (1799070.914 5395344.405 0.000, 1..."
139135,1.004662e+11,"LINESTRING Z (1799060.918 5395344.714 0.000, 1..."
139136,1.004662e+11,"LINESTRING Z (1799050.923 5395345.024 0.000, 1..."


In [209]:
rates = gpd.read_file("Data for testing/kx-nzccd-coastal-change-rates-SHP.zip")
rates

,ShrCount,LRR,LR2,LCI,LSE,EPR,EPRunc,NSM,SCE,WLR,WR2,WCI,WSE,TCD,Region,UniqueID,Start_date,End_date,Duration,geometry
0,13.0,-0.27,0.80,0.07,3.20,-0.27,0.05,-19.30,24.84,-0.23,0.65,0.09,1.37,4015.900000,Waikato,1.005336e+11,1950-04-01,2022-02-15,71,POINT (1740823.984 5718945.283)
1,9.0,0.58,0.91,0.13,5.14,0.56,0.07,39.42,39.42,0.55,0.93,0.10,1.14,44129.150000,Tasman,2.000050e+11,1950-10-11,2021-03-13,70,POINT (1573804.234 5506243.949)
2,13.0,-0.16,0.07,0.32,16.77,-0.07,0.06,-5.38,44.09,-0.19,0.08,0.34,4.94,7410.848139,Tasman,2.010001e+11,1940-02-22,2020-07-01,80,POINT (1602827.257 5444493.261)
3,13.0,-0.22,0.13,0.30,15.96,-0.12,0.06,-9.85,44.06,-0.25,0.15,0.32,4.70,7420.848139,Tasman,2.010001e+11,1940-02-22,2020-07-01,80,POINT (1602821.392 5444501.389)
4,13.0,-0.26,0.19,0.28,15.15,-0.18,0.06,-14.58,44.19,-0.29,0.21,0.30,4.41,7430.848139,Tasman,2.010001e+11,1940-02-22,2020-07-01,80,POINT (1602815.442 5444509.464)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228608,5.0,0.01,0.05,0.08,1.81,-0.02,0.07,-1.50,3.73,0.00,0.00,0.13,0.97,3223.050000,Auckland,1.010019e+11,1961-11-02,2022-12-21,61,POINT (1784449.725 5926935.288)
228609,5.0,0.06,0.63,0.07,1.45,0.03,0.07,1.91,4.73,0.05,0.36,0.09,0.71,3233.050000,Auckland,1.010019e+11,1961-11-02,2022-12-21,61,POINT (1784439.931 5926939.809)
228610,5.0,0.06,0.64,0.06,1.27,0.03,0.07,1.74,4.23,0.07,0.46,0.10,0.74,3243.050000,Auckland,1.010019e+11,1961-11-02,2022-12-21,61,POINT (1784431.172 5926944.695)
228611,5.0,0.03,0.34,0.06,1.26,0.00,0.07,0.18,2.73,0.04,0.22,0.09,0.70,3253.050000,Auckland,1.010019e+11,1961-11-02,2022-12-21,61,POINT (1784423.016 5926950.491)


In [218]:
def process_row(row):
    # This function reproduces the statistics described in https://pubs.usgs.gov/of/2021/1091/ofr20211091.pdf
    shorelines["intersect_point"] = shorelines.intersection(row.geometry)
    df = shorelines[~shorelines.intersect_point.is_empty].sort_values("Date")
    df.Date = pd.to_datetime(df.Date)
    df["YearsSinceBase"] = (df.Date - pd.Timestamp(1800, 1, 1)).dt.days / 365.25
    transect_origin = shapely.get_point(transects.geometry[0], 2)
    df["Distance"] = df.intersect_point.distance(transect_origin)
    LR = sm.OLS(df.Distance, sm.add_constant(df.YearsSinceBase)).fit()
    low, high = LR.conf_int(alpha=.1).loc["YearsSinceBase"]
    LCI = (high - low) / 2
    WLR = sm.WLS(df.Distance, sm.add_constant(df.YearsSinceBase), weights=1 / df.Total_UNCY ** 2).fit()
    low, high = WLR.conf_int(alpha=.1).loc["YearsSinceBase"]
    WCI = (high - low) / 2
    duration = (df.Date.max() - df.Date.min()).days / 365.25
    NSM = df.intersect_point.iloc[0].distance(df.intersect_point.iloc[-1])
    return pd.Series({
        "UniqueID": row.Unique_ID,
        "Region": df.Region.value_counts().idxmax(),
        "Start_date": str(df.Date.min().date()),
        "End_date": str(df.Date.max().date()),
        "Duration": round(duration),
        "ShrCount": len(df),
        "NSM": NSM,
        "SCE": df.intersect_point.apply(lambda p: df.intersect_point.distance(p).max()).max(),
        "EPR": NSM / duration,
        "EPRunc": np.sqrt(df.Total_UNCY.iloc[0] ** 2 + df.Total_UNCY.iloc[-1] ** 2) / duration,
        "LRR": LR.params.YearsSinceBase,
        "LCI": LCI,
        "LSE": np.sqrt(LR.mse_resid),
        "LR2": LR.rsquared,
        "WLR": WLR.params.YearsSinceBase,
        "WCI": WCI,
        "WSE": np.sqrt(WLR.mse_resid),
        "WR2": WLR.rsquared,
    })
pd.concat([rates[rates.UniqueID == 100522647132.0], pd.DataFrame([process_row(transects.iloc[0])])],axis=0)

,ShrCount,LRR,LR2,LCI,LSE,EPR,EPRunc,NSM,SCE,WLR,WR2,WCI,WSE,TCD,Region,UniqueID,Start_date,End_date,Duration,geometry
124687,5.0,0.240000,0.430000,0.370000,6.550000,0.300000,0.200000,15.550000,16.840000,0.280000,0.260000,0.640000,2.100000,0.0,Taranaki,1.005226e+11,1970-02-10,2022-01-14,51,POINT (1680425.889 5669776.879)
0,5.0,0.237582,0.427684,0.373421,6.553639,0.299401,0.201854,15.546738,16.839817,0.283414,0.263359,0.644026,2.098247,NaN,Taranaki,1.005226e+11,1970-02-10,2022-01-14,52,None


In [ ]:
points = gpd.read_file("Data for testing/Unique_ID_points/NorthIsland_UniqueID.shp")
points

,Unique_ID,geometry
0,1.003479e+11,POINT Z (1880322.836 5830555.229 0)
1,1.003479e+11,POINT Z (1880327.712 5830546.382 0)
2,1.003479e+11,POINT Z (1880333.202 5830537.997 0)
3,1.003479e+11,POINT Z (1880339.081 5830529.906 0)
4,1.003479e+11,POINT Z (1880345.252 5830522.035 0)
...,...,...
139133,1.004750e+11,POINT Z (1758086.752 5426055.275 0)
139134,1.004750e+11,POINT Z (1758091.623 5426064.015 0)
139135,1.004750e+11,POINT Z (1758096.813 5426072.563 0)
139136,1.004750e+11,POINT Z (1758102.158 5426081.017 0)
